In [5]:
#install.packages('caTools')

also installing the dependency 'bitops'



package 'bitops' successfully unpacked and MD5 sums checked
package 'caTools' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\amara\AppData\Local\Temp\Rtmpovbdlz\downloaded_packages


In [1]:
library(ISLR)

library(MASS)
library(glmnet)
library(caTools)
data("College")
set.seed(123)


Warning message:
"package 'MASS' was built under R version 3.6.3"Warning message:
"package 'glmnet' was built under R version 3.6.3"Loading required package: Matrix
Warning message:
"package 'Matrix' was built under R version 3.6.3"Loaded glmnet 4.1-1
Warning message:
"package 'caTools' was built under R version 3.6.3"

## Getting the college data Set and checking for null values

In [2]:
college_dats = College
sum(is.na(college_dats))

[1] 0

### No null values 

## 1 a) Dividing the data set into training and testing.

In [3]:
split = sample.split(college_dats$Apps, SplitRatio = 0.8)
training_dats = subset(college_dats, split== TRUE)
test_dats = subset(college_dats, split == FALSE)

 implementing linear model using least squares 

In [4]:
college_dats.lm = lm(Apps ~. , data= training_dats)
y_hat = predict(college_dats.lm, test_dats)

Seeing how the model faired

In [5]:
college_dats.lm_summary = summary(college_dats.lm)

Calculating the MSE

In [6]:
mean(college_dats.lm_summary$residuals^2)

[1] 1084375

## We get a very high value this tells us that the data needs to be scaled 

### Scaling the data

In [7]:
college_dats1=model.matrix(Apps ~ . -1, data = College)
college_dats[, -1] = apply(college_dats[, -1], 2, scale)
train_size =  dim(college_dats)[1] / 2
train = sample(1:dim(college_dats)[1], train_size)
test = -train

Splitting the data using train_size which is 50 percent

In [8]:
training_dats = college_dats[train, ]
test_dats = college_dats[test, ]

## implementing linear model using least squares 

In [9]:
college_dats.lm = lm(Apps ~. , data= training_dats)
y_hat = predict(college_dats.lm, test_dats)

Calculating the MSE

In [10]:
mean((test_dats[,"Apps"] - y_hat)^2)

[1] 0.07993743

## We got an error of 7.993 percent 

## 1 b) Ridge Regression Model

In [11]:
test_mod = model.matrix(Apps ~ . -1, data = test_dats)
train_mod =  model.matrix(Apps ~ . -1 , data = training_dats)
ridge_model =  cv.glmnet(train_mod, training_dats[, "Apps"], alpha = 0)

finding the best lambda value for our data which is the min value

In [12]:
lamda_best = ridge_model$lambda.min
lamda_best

[1] 0.1007566

## best lambda is 0.100756

In [13]:
ridge_model_pred = predict(ridge_model, s=lamda_best, newx= test_mod)
mean((test_dats[, "Apps"] - ridge_model_pred)^2)

[1] 0.06667753

## MSE error 6.6677

## 1 d) Fit a lasso model on the training set, with λ chosen by cross-validation.Report the test error obtained, along with the number of non-zero coefficient estimates.

Lets find the best lambda value for lasso 

In [14]:
lasso_model =  cv.glmnet(train_mod,training_dats[, "Apps"], alpha = 1)
lambda_best = lasso_model$lambda.min
lambda_best

[1] 0.0005377077

## The best lambda value is  0.0005377077

Predicting using lasso model

In [15]:
lasso_model_pred = predict(lasso_model,s=lambda_best,newx = test_mod)
mean((test_dats[, "Apps"] - lasso_model_pred)^2)

[1] 0.07926623

## MSE error 7.926

finding Coefficients of the lasso model

In [16]:
## finding Coefficients of the lasso model
lasso_model <-  glmnet(college_dats1, College[, "Apps"], alpha = 1)
predict(lasso_model, s = lambda_best, type = "coefficients")

19 x 1 sparse Matrix of class "dgCMatrix"
                        1
(Intercept) -962.43857204
PrivateNo    491.04485135
PrivateYes     .         
Accept         1.57033288
Enroll        -0.75961467
Top10perc     48.14698891
Top25perc    -12.84690694
F.Undergrad    0.04149116
P.Undergrad    0.04438973
Outstate      -0.08328388
Room.Board     0.14943472
Books          0.01532293
Personal       0.02909954
PhD           -8.39597537
Terminal      -3.26800340
S.F.Ratio     14.59298267
perc.alumni   -0.04404771
Expend         0.07712632
Grad.Rate      8.28950241